<a href="https://colab.research.google.com/github/hnarges91/violence-detection/blob/main/violence_detection_final_version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>







# **mount drive**

In [1]:
# %tensorflow_version 1.x

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/My Drive/Kaggle/datasets/

/content/gdrive/My Drive/Kaggle/datasets


In [4]:
%cd /content/gdrive/My Drive/Kaggle/features/

[Errno 2] No such file or directory: '/content/gdrive/My Drive/Kaggle/features/'
/content/gdrive/My Drive/Kaggle/datasets


In [5]:
# !pip install keras-rl2

# **reset all variavle value**

In [6]:
#setup
#reset variavle value
from IPython import get_ipython
get_ipython().magic('reset -sf')
#intellisence
%config IPCcompleter.greedy = True
import numpy as np
np.random.seed(400)  # for reproducibility

# **import library**

In [7]:
# import cv2
# #import math
# import os
# import time
# import numpy as np
# from numpy import asarray
# from numpy import clip
# import random

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input


import keras , tensorflow
from keras import models
from keras.models import Model
from keras import optimizers
# from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing import image
# from keras import applications
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# from tensorflow.keras.utils import load_img, img_to_array
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.layers import Input
# from keras.utils import to_categorical
# from keras.utils.vis_utils import model_to_dot
# from keras.layers import BatchNormalization
# from keras import regularizers
# from keras.regularizers import l1
# from keras.regularizers import l2
# from IPython.display import SVG


# import matplotlib.pyplot as plt
# from PIL import Image
# from PIL import Image, ImageEnhance
# %matplotlib inline

# #import pandas as pd
# from sklearn.svm import SVC
# from sklearn import svm
# from sklearn.model_selection import GridSearchCV, train_test_split
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.metrics import accuracy_score


# from keras.models import Sequential
# from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D,AveragePooling1D
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn.datasets import load_iris
# from numpy import unique

# ***Pre_train Models***

In [8]:
from keras.layers import Input
from keras.models import Model
from keras.applications import VGG16, VGG19, ResNet50

def choose_model(model_name):
    """
    Choose a pre-trained convolutional neural network model for feature extraction.

    Parameters:
    - model_name: A string specifying the name of the model to use ('vgg16', 'vgg19', or 'resnet50').

    Returns:
    - model: The chosen pre-trained model.
    """

    # Define the input tensor shape
    input_tensor = Input(shape=(224, 224, 3))

    if model_name == 'vgg16':
        # Load VGG16 model pre-trained on ImageNet and remove the top layers
        model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor, pooling='avg')
        # Define a custom model that outputs features extracted by the last convolutional layer
        custom_model = Model(model.inputs, model.layers[-2].output)

    elif model_name == 'vgg19':
        # Load VGG19 model pre-trained on ImageNet and remove the top layers
        model = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor, pooling='avg')
        # Define a custom model that outputs features extracted by the second-to-last convolutional layer
        custom_model = Model(model.inputs, model.layers[-3].output)

    elif model_name == 'resnet50':
        # Load ResNet50 model pre-trained on ImageNet and remove the top layers
        model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor, pooling='avg')
        # Define a custom model that outputs features extracted by the last convolutional layer
        custom_model = Model(model.inputs, model.layers[-2].output)

    return model


# ***feature extraction and Labeling***

In [9]:
import os
import cv2
import numpy as np
import tensorflow
# from tensorflow.keras.applications.vgg16 import preprocess_input

def extract_frame_features(frames, custom_model):

    """
    Extract features from frames using a pretrained model.

    Parameters:
    - frames: Input frames as a list of numpy arrays.
    - custom_model: Pretrained model for feature extraction.

    Returns:
    - frame_features: Features extracted from the frames as a numpy array.
    """

    # Resize all frames to match the input size of the pretrained model
    frames_resized = [cv2.resize(frame, (224, 224)) for frame in frames]

    # Convert the resized frames to a numpy array
    img_data = np.asarray(frames_resized)


    # Preprocess the input data according to the preprocessing function used for the pretrained model
    img_data = preprocess_input(img_data)


    # Extract features from the frames using the pretrained model
    frame_features = custom_model.predict(img_data)

    return frame_features

In [10]:
def extract_video_frames(video_dir, model_name, frame_rate=1):
    """
    Extract spatial features from videos using a pretrained model.

    Parameters:
    - video_dir: Directory containing the input videos.
    - model_name: Name of the model.
    - frame_rate: Rate at which frames will be extracted (e.g., every nth frame).

    Returns:
    - feature_list: List containing extracted features for all videos.
    """
    custom_model = choose_model(model_name)
    # custom_model.summary()
    feature_list = []
    video_counter = 0
    videos = sorted([vfile for vfile in os.listdir(video_dir)])
    print(" number of videos",len(videos))
    for video in videos:
        if video.endswith(".mp4") or video.endswith(".avi"):  # Check if file is a video
            video_counter += 1
            print(f"Video {video_counter}: {video}")
            video_path = os.path.join(video_dir, video)
            cap = cv2.VideoCapture(video_path)

            frames = []
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            # frame_step = int(fps / frame_rate)
            frame_step = 1


            for i in range(0, frame_count, frame_step):
                cap.set(cv2.CAP_PROP_POS_FRAMES, i)  # Set frame position
                ret, frame = cap.read()  # Read frame
                if ret:
                    frames.append(frame)

            cap.release()
            # Extract features for all frames of the video
            video_features = extract_frame_features(frames, custom_model)
            print ("feature shape ",np.shape(video_features))
            feature_list.append(video_features)
            print ("feature_list:",np.shape(feature_list))
    return feature_list


In [11]:

import tensorflow.keras.backend as K

def labeling(violence_features, nonviolence_features):
    """
    Label the violence and non-violence videos.

    Parameters:
    - violence_features: Features extracted from violence videos.
    - nonviolence_features: Features extracted from non-violence videos.

    Returns:
    - X_feature: Combined features for testing.
    - y_label: Combined labels for testing.
    """

    K.clear_session()

    print("********** Labeling **********")

    # Label for violence videos
    violence_label_test = np.zeros((violence_features.shape[0], 1))  # Label 0 for violence

    # Label for non-violence videos
    nonviolence_label_test = np.ones((nonviolence_features.shape[0], 1))  # Label 1 for non-violence

    # Combine features and labels
    features_combined = np.concatenate((violence_features, nonviolence_features), axis=0)
    labels_combined = np.concatenate((violence_label_test, nonviolence_label_test))

    print("Features combined shape:", features_combined.shape)

    # Create an array of indices and shuffle it
    indices = np.arange(features_combined.shape[0])
    np.random.shuffle(indices)

    # Shuffle the combined array using the shuffled indices
    X_feature = features_combined[indices]
    y_label = labels_combined[indices]

    return X_feature, y_label


# **load features matrix or extraxt feature**

In [12]:


def load_feature_file(file_name):
    """
    Load features from a numpy file if it exists.

    Parameters:
    - file_name: Name of the file to load features from.

    Returns:
    - features: Loaded features if the file exists, otherwise None.
    """
    if os.path.isfile(file_name):
        print("File exists")
        return np.load(file_name, allow_pickle=True)
    else:
        print("File does not exist")
        return None

In [13]:


def feature_loader(dataset_name, model_name):
    """
    Load features for violence and non-violence videos from the specified dataset and model.

    Parameters:
    - dataset_name: Name of the dataset.
    - model_name: Name of the feature extraction model.

    Returns:
    - violent_features: Features extracted from violence videos.
    - nonviolent_features: Features extracted from non-violence videos.
    """
    try:
        # Load features for violence videos
        label = 0
        class_name = 'violent_feature'
        # Change directory to where features are stored
        %cd /content/gdrive/My Drive/Kaggle/features/
        # Get paths for violence and non-violence frames
        v_path_frame, nv_path_frame = dataset_address(dataset_name)

        violent_file_name = f"{dataset_name}_{class_name}_{model_name}.npy"
        print(violent_file_name)

        violent_features = load_feature_file(violent_file_name)

        # If features are not loaded, extract features
        if violent_features is None:
            violent_features = extract_video_frames(v_path_frame, model_name, frame_rate=1)

            # Save extracted features to a file
            np.save(violent_file_name, violent_features)

            # Print shape of violent features
            print("--------------- Features Shape -------------")
            print("violent_feature =", np.shape(violent_features))
            print("---------------------------------------------" * 3)

    except Exception as e:
        print("Error loading violent features:", e)

    try:
        # Load features for non-violence videos
        label = 1
        class_name = 'nonviolent_feature'
        nonviolent_file_name = f"{dataset_name}_{class_name}_{model_name}.npy"

        nonviolent_features = load_feature_file(nonviolent_file_name)

        # If features are not loaded, extract features
        if nonviolent_features is None:
            nonviolent_features = extract_video_frames(nv_path_frame, model_name, frame_rate=1)

            # Save extracted features to a file
            np.save(nonviolent_file_name, nonviolent_features)

            # Print shape of non-violent features
            print("--------------- Features Shape -------------")
            print("nonviolent_feature =", np.shape(nonviolent_features))
            print("---------------------------------------------" * 3)

    except Exception as e:
        print("Error loading nonviolent features:", e)

    return violent_features, nonviolent_features


In [14]:
  """
    Returns the paths for violence and non-violence frames based on the specified dataset name.

    Parameters:
    - dataset_name: Name of the dataset.

    Returns:
    - v_path_frame: Path to violence frames.
    - nv_path_frame: Path to non-violence frames.
  """

'\n  Returns the paths for violence and non-violence frames based on the specified dataset name.\n\n  Parameters:\n  - dataset_name: Name of the dataset.\n\n  Returns:\n  - v_path_frame: Path to violence frames.\n  - nv_path_frame: Path to non-violence frames.\n'

In [15]:


def dataset_address(dataset_name):
  if dataset_name == "hockey":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/hockey_dataset/HockeyFights"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/hockey_dataset/HockeyFights_no"
  elif dataset_name == "Movies_Fight":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/Movies_Fight_Detection_Dataset/fights"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/Movies_Fight_Detection_Dataset/noFights"
  elif dataset_name == "violent_flow_crowd":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/violent_flow_crowd/Violence"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/violent_flow_crowd/NonViolence"
  elif dataset_name == "Real_Life_Violence_Dataset":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/Real_Life_Violence_Dataset/Violence"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/Real_Life_Violence_Dataset/NonViolence"
  elif dataset_name == "RWF":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF/train/Fight"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF/train/NonFight"
  elif dataset_name == "RWF_reconstructed":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_reconstructed/train/Fight"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_reconstructed/train/NonFight"
  elif dataset_name == "RWF_window_5":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_window_5/train/Fight"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_window_5/train/NonFight"
  elif dataset_name == "RWF_window_10":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_window_10/train/Fight"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_window_10/train/NonFight"
  elif dataset_name == "RWF_window_15":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_window_15/train/Fight"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_window_15/train/NonFight"
  elif dataset_name == "RWF_window_20":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_window_20/train/Fight"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/RWF_Residuals/RWF_window_20/train/NonFight"
  elif dataset_name == "reconstructed_mat2gray":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/hockey_dataset_reconstructed_mat2gray/HockeyFights"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/hockey_dataset_reconstructed_mat2gray/HockeyFights_no"
  elif dataset_name == "reconstructed_mat2gray(abs(res))":
    v_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/hockey_dataset_reconstructed_mat2gray(abs(res))/HockeyFights"
    nv_path_frame = "/content/gdrive/My Drive/Kaggle/datasets/hockey_dataset_reconstructed_mat2gray(abs(res))/HockeyFights_no"
  elif dataset_name == "hockey_dataset_rec":
    v_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/res/hockey_dataset_rec/HockeyFights"
    nv_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/res/hockey_dataset_rec/HockeyFights_no"
  elif dataset_name == "Movies_Fight_rec":
    v_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/res/movie_fight_rec/fights"
    nv_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/res/movie_fight_rec/noFights"
  elif dataset_name == "violent_flow_crowd_rec":
    v_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/res/violent_flow_crowd_rec/Violence"
    nv_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/res/violent_flow_crowd_rec/NonViolence"
  elif dataset_name == "real_lif_accumulated_results5":
    v_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_lif_accumulated_results/real_life_Similarity_win5/Violence"
    nv_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_lif_accumulated_results/real_life_Similarity_win5/NonViolence"
  elif dataset_name == "real_lif_accumulated_results10":
    v_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_lif_accumulated_results/real_life_Similarity_win10/Violence"
    nv_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_lif_accumulated_results/real_life_Similarity_win10/NonViolence"
  elif dataset_name == "real_lif_accumulated_results15":
    v_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_lif_accumulated_results/real_life_Similarity_win15/Violence"
    nv_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_lif_accumulated_results/real_life_Similarity_win15/NonViolence"
  elif dataset_name == "real_lif_accumulated_results20":
    v_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_lif_accumulated_results/real_life_Similarity_win20/Violence"
    nv_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_lif_accumulated_results/real_life_Similarity_win20/NonViolence"
  elif dataset_name == "real_life_reconstructed":
    v_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_life_reconstructed/Violence"
    nv_path_frame = "/content/gdrive/MyDrive/Kaggle/datasets/real_life_reconstructed/NonViolence"
    # Add more cases for other dataset names as needed
  else:
      # Handle the case where dataset_name is not recognized
      print("Dataset name not recognized")
      v_path_frame = ""
      nv_path_frame = ""

  return v_path_frame,nv_path_frame

# **dataset**
# **split to train , test , val**
# **choose between diffrent dataset**

In [16]:
# from sklearn.model_selection import train_test_split

# def split_data(features, train_size):
#     """
#     Split the data into training and testing sets for each class separately, ensuring equal sizes of each class in both sets.

#     Parameters:
#     - features: List of feature arrays.
#     - train_size: Size of the training set relative to the total size of the dataset.

#     Returns:
#     - train_part: Training set for each class.
#     - test_part: Testing set for each class.
#     """

#     # Calculate the size of the testing set
#     test_size = 1 - train_size

#     # Split the data into training and testing sets
#     train_part, test_part = train_test_split(features, train_size=train_size, test_size=test_size, random_state=1, shuffle=True)

#     return train_part, test_part


# ***PMF***

In [17]:


def pooling_feature(features, pooling_type):
    """
    Perform pooling operation on the input features.

    Parameters:
    - features: List of feature arrays.
    - pooling_type: Type of pooling operation to perform ("mean" or "max").

    Returns:
    - pooled_features: Pooled features.
    """

    # Initialize list to store pooled features
    pooled_features = []

    # Perform pooling operation based on the specified type
    for feature_array in features:
        if pooling_type == "mean":
            pooled_feature = np.mean(feature_array, axis=0)  # Calculate mean along each feature dimension
        elif pooling_type == "max":
            pooled_feature = np.max(feature_array, axis=0)  # Calculate max along each feature dimension
        else:
            raise ValueError("Invalid pooling_type. Use 'mean' or 'max'.")

        pooled_features.append(pooled_feature)

    # Convert the list of pooled features to a numpy array
    pooled_features = np.array(pooled_features)

    return pooled_features


# ***Classification***
# **SVM**
# Neural network
# results

In [18]:
#SVM classification
def SVM_Classification(X_train,y_train,X_test,y_test):
  svc = svm.SVC(kernel='linear').fit(X_train, y_train)
  rbf_svc = svm.SVC(kernel='rbf', gamma=0.7).fit(X_train, y_train)
  poly_svc = svm.SVC(kernel='poly', degree=3).fit(X_train, y_train)
  lin_svc = svm.LinearSVC().fit(X_train, y_train)
  y_pred = svc.predict(X_test)
  y_rbf_svc=rbf_svc.predict(X_test)
  y_poly_svc=poly_svc.predict(X_test)
  y_lin_svc=lin_svc.predict(X_test)

  print("-----SVM Clasiification---------")
  print(accuracy_score(y_test, y_pred))
  print("-----SVM Clasiification---------")
  print(classification_report(y_test,y_pred))
  print("-----------------------"*3)
  print(classification_report(y_test,y_pred))
  print(classification_report(y_test,y_rbf_svc))
  print(classification_report(y_test,y_poly_svc))
  print(classification_report(y_test,y_lin_svc))


In [19]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [20]:
def neural_network_classification(X_train, y_train, input_dim):
    """
    Perform neural network classification with a fully connected network having 8 layers.

    Parameters:
    - X_train: Input training features.
    - y_train: Training labels.
    - input_dim: Dimension of the input features.

    Returns:
    - history: Training history of the model.
    - model: Trained neural network model.
    """

    # Convert labels to categorical
    y_train = to_categorical(y_train)

    # Build neural network
    model = models.Sequential()
    model.add(Dense(512, activation='relu', input_shape=input_dim))  # Layer 1
    model.add(Dense(512, activation='relu'))  # Layer 2
    model.add(Dense(512, activation='relu'))  # Layer 3
    model.add(Dropout(0.2))  # Regularization for layer 3
    model.add(Dense(512, activation='relu'))  # Layer 4
    model.add(Dropout(0.3))  # Regularization for layer 4
    model.add(Dense(512, activation='relu'))  # Layer 5
    model.add(Dropout(0.3))  # Regularization for layer 5
    model.add(Dense(2, activation='softmax'))  # Output layer

    # Define early stopping callback
    callback = keras.callbacks.EarlyStopping(
        monitor="loss", min_delta=0, patience=6, verbose=1,
        mode="min", baseline=None, restore_best_weights=True
    )

    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=5e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train model
    history = model.fit(X_train, y_train, epochs=32, verbose=0,
                        shuffle=True, validation_split=0.2, callbacks=[callback])

    return history, model


In [21]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

def result(x, y, history, model):
    """
    Print classification report, confusion matrix, and plot accuracy and loss.

    Parameters:
    - x: Test features.
    - y: True labels.
    - history: History object returned by model.fit().
    - model: Trained neural network model.
    """

    print("+++ Generating result... +++")

    # Predict labels
    pred = model.predict(x)
    # Evaluate model
    score = model.evaluate(x, y, verbose=0)
    print("------------- Neural network classification -------------")
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    real = []
    prediction = []

    # Convert predicted probabilities to labels
    for p in pred:
        if p[0] >= .5:
            prediction.append(0)
        elif p[1] >= .5:
            prediction.append(1)
    # Convert true labels to binary labels
    for i in y:
        if i[0] >= .5:
            real.append(0)
        elif i[1] >= .5:
            real.append(1)

    print("---------------------- Classification report ----------------------")
    print(classification_report(real, prediction))

    print("---------------------- Confusion Matrix ----------------------")
    conf_mat = confusion_matrix(real, prediction)
    print(conf_mat)

    print("--------- Plot accuracy and loss ------","\n")

    # Plot accuracy and loss
    plt.figure(figsize=(12,4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')

    plt.show()


# ***Violence Detection***

In [22]:
import time
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

def violence_detection(model_name, dataset_name, pooling_type):
    """
    Perform violence detection using a specified model, dataset, and pooling type.

    Parameters:
    - model_name: Name of the pre-trained model to use.
    - dataset_name: Name of the dataset.
    - pooling_type: Type of pooling operation to perform ("mean" or "max").

    Returns:
    - accuracy: Accuracy of the violence detection model.
    - loss: Loss of the violence detection model.
    """

    keras.backend.clear_session()

    # Extract features from pre-trained model
    violent_feature, nonviolent_feature = feature_loader(dataset_name, model_name)

    # Perform classification

    data, label = labeling(violent_feature, nonviolent_feature)
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

    # pmf
    X_train = pooling_feature(X_train, pooling_type)
    X_test = pooling_feature(X_test, pooling_type)

    input_dim= (np.shape(X_train)[1],np.shape(X_train)[2])
    # input_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3])

    start = time.process_time()
    history, model = neural_network_classification(X_train, y_train, input_dim)
    print("Process time:", time.process_time() - start)

    X_test = np.array(X_test)

    # Predict
    y = to_categorical(y_test)
    pred = model.predict(X_test)
    score = model.evaluate(X_test, y, verbose=0)

    result(X_test, y, history, model)

    # Clean up
    del X_train, y_train, X_test, y_test, history, model

    return score[1], score[0]


# ***Main***

In [23]:
"""
  dataset_name == "hockey": original hockey dataset
  dataset_name == "hockey_dataset_rec": reconstructed hockey dataset

  dataset_name == "Movies_Fight":
  dataset_name == "Movies_Fight_rec":

  dataset_name == "violent_flow_crowd":
  dataset_name == "violent_flow_crowd_rec":

  dataset_name == "RWF":
  dataset_name == "RWF_reconstructed":

  dataset_name == "RWF_window_5":
  dataset_name == "RWF_window_10":
  dataset_name == "RWF_window_15":
  dataset_name == "RWF_window_20":


  dataset_name == "kaggle":
  dataset_name == "real_life_reconstructed":

  dataset_name == "real_lif_accumulated_results5":
  dataset_name == "real_lif_accumulated_results10":
  dataset_name == "real_lif_accumulated_results15":
  dataset_name == "real_lif_accumulated_results20":


  dataset_name == "reconstructed_mat2gray":
  dataset_name == "reconstructed_mat2gray(abs(res))":
"""

'\n  dataset_name == "hockey": original hockey dataset\n  dataset_name == "hockey_dataset_rec": reconstructed hockey dataset\n\n  dataset_name == "Movies_Fight":\n  dataset_name == "Movies_Fight_rec":\n\n  dataset_name == "violent_flow_crowd":\n  dataset_name == "violent_flow_crowd_rec":\n\n  dataset_name == "RWF":\n  dataset_name == "RWF_reconstructed":\n\n  dataset_name == "RWF_window_5":\n  dataset_name == "RWF_window_10":\n  dataset_name == "RWF_window_15":\n  dataset_name == "RWF_window_20":\n\n\n  dataset_name == "kaggle":\n  dataset_name == "real_life_reconstructed":\n\n  dataset_name == "real_lif_accumulated_results5":\n  dataset_name == "real_lif_accumulated_results10":\n  dataset_name == "real_lif_accumulated_results15":\n  dataset_name == "real_lif_accumulated_results20":\n\n\n  dataset_name == "reconstructed_mat2gray":\n  dataset_name == "reconstructed_mat2gray(abs(res))":\n'

In [24]:
#Initialization
model_name ='vgg16'#model can choose between  = 'vgg16','vgg19','resnet50'
dataset_name = "hockey"
pooling_type="mean" # can choose between max , mean for pooling


In [25]:
losss=[]
accs=[]
tag=0 # for print neural network result for 1 time and svm classification
for i in range (0,50):
  acc,loss= violence_detection(model_name, dataset_name, pooling_type)
  accs.append(acc)
  losss.append(loss)


print("-------------------")
print("+++ mean ... +++")
acc_mean= np.mean(accs)
loss_mean=np.mean(losss)
print(acc_mean)
print(loss_mean)

print("-------------------")
print("+++ median ... +++")
median_acc = np.median(accs)
median_loss = np.median(losss)
print(median_acc)
print(median_loss)

[Errno 2] No such file or directory: '/content/gdrive/My Drive/Kaggle/features/'
/content/gdrive/My Drive/Kaggle/datasets
hockey_violent_feature_vgg16.npy
File does not exist
58889256/58889256 [==============================] - 4s 0us/step
 number of videos 502
Video 1: (1).avi
2/2 [==============================] - 10s 2s/step
feature shape  (41, 512)
feature_list: (1, 41, 512)
Video 2: (10).avi
2/2 [==============================] - 0s 99ms/step
feature shape  (41, 512)
feature_list: (2, 41, 512)
Video 3: (100).avi
2/2 [==============================] - 0s 99ms/step
feature shape  (41, 512)
feature_list: (3, 41, 512)
Video 4: (101).avi
2/2 [==============================] - 0s 99ms/step
feature shape  (41, 512)
feature_list: (4, 41, 512)
Video 5: (102).avi
2/2 [==============================] - 0s 99ms/step
feature shape  (41, 512)
feature_list: (5, 41, 512)
Video 6: (103).avi
2/2 [==============================] - 0s 102ms/step
feature shape  (41, 512)
feature_list: (6, 41, 512)
Vid

AttributeError: 'NoneType' object has no attribute 'shape'